# Working with Data Frames
## Introducing the Data Frame
- The dataset has four fields: `Name`, `Date`, `Grade` and `Price`.
- Any variable in a dataset can be classified as either a ***numerical variable*** or ***categorical vairable***.
  - Categorical vairbles can be further split into ***nominal variables***, ***ordinal variables***.

In [2]:
using DataFrames, CSV
data = CSV.read("../data/purchaseData.csv", DataFrame, copycols = true)

println(size(data), "\n")
println(names(data), "\n")
println(first(data,6), "\n")
println(describe(data), "\n")

(200, 4)

["Name", "Date", "Grade", "Price"]

6×4 DataFrame
 Row │ Name       Date        Grade     Price   
     │ String15?  String15?   String1?  Int64?  
─────┼──────────────────────────────────────────
   1 │ MARYANNA   14/09/2008  A           79700
   2 │ REBECCA    11/03/2008  B         missing 
   3 │ ASHELY     5/08/2008   E           24311
   4 │ KHADIJAH   2/09/2008   missing     38904
   5 │ TANJA      1/12/2008   C           47052
   6 │ JUDIE      17/05/2008  D           34365

4×7 DataFrame
 Row │ variable  mean     min        median   max        nmissing  eltype                   
     │ Symbol    Union…   Any        Union…   Any        Int64     Union                    
─────┼──────────────────────────────────────────────────────────────────────────────────────
   1 │ Name               ABBEY               ZACHARY          17  Union{Missing, String15}
   2 │ Date               1/07/2008           9/10/2008         4  Union{Missing, String15}
   3 │ Grade              

In [3]:
sum([ismissing(i) for i in data.Name])

17

## Referencing Data

In [4]:
using DataFrames, CSV
data = CSV.read("../data/purchaseData.csv", DataFrame, copycols = true)

println("Grade of person 1: ", data[1,3],
    ", ", data[1,:Grade],
    ", ", data.Grade[1], "\n")
println(data[[1,2,4],:], "\n")
println(data[13:15, :Name], "\n")
println(data.Name[13:15], "\n")
println(data[13:15, [:Name]])

Grade of person 1: A, A, A

3×4 DataFrame
 Row │ Name       Date        Grade     Price   
     │ String15?  String15?   String1?  Int64?  
─────┼──────────────────────────────────────────
   1 │ MARYANNA   14/09/2008  A           79700
   2 │ REBECCA    11/03/2008  B         missing 
   3 │ KHADIJAH   2/09/2008   missing     38904

Union{Missing, String15}["SAMMIE", missing, "STACEY"]

Union{Missing, String15}["SAMMIE", missing, "STACEY"]

3×1 DataFrame
 Row │ Name      
     │ String15? 
─────┼───────────
   1 │ SAMMIE
   2 │ missing   
   3 │ STACEY


## Modifying Data
- Functions performed on a data frame will return a copy of that data frame. In other words, no underlying change will be made to the data frame object, but rather a shallow copy will be made and returned as output.
- By default, the columns of a data frame are ***immutable***, which means that the values within them cannot be changed. In order to make changes to a column, the column must first be mutable. One way to do this is by including `copycols=true` when a data frame is created from a `csv` file. This argument has the effect of making all columns mutable. Another way is by using `!` when referencing the rows of a data frame. For example, `df[!, :X]` references the underlying data in column `:X`, while `df[:,:X]` simply references a shallow copy.

In [5]:
using DataFrames, CSV
data1 = CSV.read("../data/purchaseData.csv", DataFrame)
data2 = CSV.read("../data/purchaseData.csv", DataFrame, copycols=true)

try data1[1,:name] = "YARDEN" catch; @warn "Cannot: data1 is immutable" end;

data2[1,:Name] = "YARDEN"
println("\n",first(data2,3),"\n")

data1[!, :Price] ./= 1000
rename!(data1, :Price=>Symbol("Price(000's)"))
println(first(data1,3), "\n")

replace!(data2[!,:Grade], ["E"=>"F", "D"=>"E"]...)
println(first(data2,3),"\n")


3×4 DataFrame
 Row │ Name       Date        Grade     Price   
     │ String15?  String15?   String1?  Int64?  
─────┼──────────────────────────────────────────
   1 │ YARDEN     14/09/2008  A           79700
   2 │ REBECCA    11/03/2008  B         missing 
   3 │ ASHELY     5/08/2008   E           24311



┌ Warning: Cannot: data1 is immutable
└ @ Main In[5]:5


3×4 DataFrame
 Row │ Name       Date        Grade     Price(000's) 
     │ String15?  String15?   String1?  Float64?     
─────┼───────────────────────────────────────────────
   1 │ MARYANNA   14/09/2008  A               79.7
   2 │ REBECCA    11/03/2008  B          missing     
   3 │ ASHELY     5/08/2008   E               24.311

3×4 DataFrame
 Row │ Name       Date        Grade     Price   
     │ String15?  String15?   String1?  Int64?  
─────┼──────────────────────────────────────────
   1 │ YARDEN     14/09/2008  A           79700
   2 │ REBECCA    11/03/2008  B         missing 
   3 │ ASHELY     5/08/2008   F           24311



## Copying a Data Frame

In [6]:
using DataFrames, CSV
data1 = CSV.read("../data/purchaseData.csv", DataFrame, copycols=true)
println("Original value: ", data1.Name[1], "\n")

data2 = data1
data2.Name[1] = "EMILY"
@show data1.Name[1]

data1 = CSV.read("../data/purchaseData.csv", DataFrame, copycols=true)
data2 = copy(data1)
data2.Name[1] = "EMILY"
@show data1.Name[1]
println()

data1 = DataFrame()
data1.X = [[0,1], [100, 101]]
data2 = copy(data1)
data2.X[1][1] = -1
@show data1.X[1][1]

data1.X = [[0,1], [100, 101]]
data2 = deepcopy(data1)
data2.X[1][1] = -1
@show data1.X[1][1];

Original value: MARYANNA

data1.Name[1] = "EMILY"
data1.Name[1] = "MARYANNA"

(data1.X[1])[1] = -1
(data1.X[1])[1] = 0


## Handling Missing Entries
- `skipmissing()`: skip all `missing` values
- `coalesce.(dfc::DataFrameColumns, string::String)`: replace all `missing` with string
- `dropmissing(df::DataFrame, colName::Symbol)`: drop all cases which `colName` is `missing` value
- `completecases(df::DataFrame)`: judge whether a case contains `missing` or not

In [7]:
using Statistics, DataFrames, CSV
data = CSV.read("../data/purchaseData.csv", DataFrame, copycols=true)

println(mean(data.Price),"\n")
println(mean(skipmissing(data.Price)), "\n")
println(coalesce.(data.Grade, "QQ")[1:4], "\n")
println(first(dropmissing(data, :Price), 4), "\n")
println(sum(ismissing.(data.Name)), "\n")
println(findall(completecases(data))[1:4])

missing

39702.01075268817

AbstractString["A", "B", "E", "QQ"]

4×4 DataFrame
 Row │ Name       Date        Grade     Price 
     │ String15?  String15?   String1?  Int64 
─────┼────────────────────────────────────────
   1 │ MARYANNA   14/09/2008  A         79700
   2 │ ASHELY     5/08/2008   E         24311
   3 │ KHADIJAH   2/09/2008   missing   38904
   4 │ TANJA      1/12/2008   C         47052

17

[1, 3, 5, 6]


In [8]:
t = [missing, missing, missing, 1]
coalesce.(t, "a")

4-element Vector{Any}:
  "a"
  "a"
  "a"
 1

In [9]:
completecases(data)

200-element BitVector:
 1
 0
 1
 0
 1
 1
 1
 1
 0
 1
 1
 0
 1
 ⋮
 1
 1
 1
 1
 1
 0
 1
 1
 1
 1
 1
 1

## Reshaping, Joining, and Manipulating Data Frames
- Joins' style like SQL
  - `innerjoin()`
  - `outerjoin()`
  - `rightjoin()`
  - `leftjoin()`
![SQL Joins](https://cdn.jsdelivr.net/gh/BNUPSYCHER/picture-host/SQL_Joins.svg)

In [10]:
using Statistics, DataFrames, CSV
data = CSV.read("../data/purchaseData.csv", DataFrame, copycols=true)

newCol = DataFrame(Validated=ones(Int, size(data,1)))
newRow = DataFrame([["JOHN", "JACK"] [123456, 909595]],[:Name, :PhoneNo])
newData = DataFrame(Name=["JOHN", "ASHELY", "MARYANNA"],
    Job=["Lawyer", "Doctor", "Lawyer"])

data = hcat(data, newCol)
println(first(data,3), "\n")

data = vcat(data, newRow, cols=:union)
println(last(data,3), "\n")

data = innerjoin(data, newData, on=:Name, matchmissing=:equal)
println(data,"\n")

select!(data, [:Name, :Job])
println(data, "\n")

unique!(data, :Job)
println(data, "\n")

3×5 DataFrame
 Row │ Name       Date        Grade     Price    Validated 
     │ String15?  String15?   String1?  Int64?   Int64     
─────┼─────────────────────────────────────────────────────
   1 │ MARYANNA   14/09/2008  A           79700          1
   2 │ REBECCA    11/03/2008  B         missing          1
   3 │ ASHELY     5/08/2008   E           24311          1

3×6 DataFrame
 Row │ Name  Date        Grade     Price    Validated  PhoneNo 
     │ Any   String15?   String1?  Int64?   Int64?     Any     
─────┼─────────────────────────────────────────────────────────
   1 │ RIVA  30/12/2008  E           21842          1  missing 
   2 │ JOHN  missing     missing   missing    missing  123456
   3 │ JACK  missing     missing   missing    missing  909595

3×7 DataFrame
 Row │ Name      Date        Grade     Price    Validated  PhoneNo  Job    
     │ Any       String15?   String1?  Int64?   Int64?     Any      String 
─────┼─────────────────────────────────────────────────────────────

## Useful Operations for Data Frames

In [11]:
using DataFrames, CSV, Dates, Statistics, CategoricalArrays
data = dropmissing(CSV.read("../data/purchaseData.csv", DataFrame, copycols=true))

data[!, :Date] = Date.(data[!,:Date], "d/m/y")
println(first(sort(data, :Date), 3), "\n")

println(first(filter(row -> row[:Price] > 5000, data), 3), "\n")

data[!,:Grade] = categorical(data[!,:Grade])
println(first(data, 3), "\n")

println(
    combine(groupby(data, :Grade), 
    :Price => (x->length(x)) => :NumSold, 
    :Price=> (x->mean(x)) => :AvgPrice)
)

3×4 DataFrame
 Row │ Name       Date        Grade    Price 
     │ String15   Date        String1  Int64 
─────┼───────────────────────────────────────
   1 │ STEPHEN    2008-02-11  D        33155
   2 │ JACKELINE  2008-02-12  E         8257
   3 │ ARDELL     2008-03-03  C        46911

3×4 DataFrame
 Row │ Name      Date        Grade    Price 
     │ String15  Date        String1  Int64 
─────┼──────────────────────────────────────
   1 │ MARYANNA  2008-09-14  A        79700
   2 │ ASHELY    2008-08-05  E        24311
   3 │ TANJA     2008-12-01  C        47052

3×4 DataFrame
 Row │ Name      Date        Grade  Price 
     │ String15  Date        Cat…   Int64 
─────┼────────────────────────────────────
   1 │ MARYANNA  2008-09-14  A      79700
   2 │ ASHELY    2008-08-05  E      24311
   3 │ TANJA     2008-12-01  C      47052

5×3 DataFrame
 Row │ Grade  NumSold  AvgPrice 
     │ Cat…   Int64    Float64  
─────┼──────────────────────────
   1 │ A           15   76606.7
   2 │ B       

## A Cleaning and Imputation Example
- Instead of simply deleting rows, one way of dealing with `missing` values is to use ***imputation***. This involves substituting missing entries with velues based on either the data observed or according to some other logic.
- Example

In [12]:
using DataFrames, CSV, Statistics
data = CSV.read("../data/purchaseData.csv", DataFrame)

rowsKeep = .!(ismissing.(data.Grade) .& ismissing.(data.Price))
data = data[rowsKeep, :]

replace!(x -> ismissing(x) ? "QQ" : x, data.Name)
replace!(x -> ismissing(x) ? "31/06/2008" : x, data.Date)

grPr = combine(groupby(dropmissing(data) ,:Grade),
    :Price => (x -> round(mean(x), digits=-3)) => :AvgPrice)

d = Dict(grPr[:,1] .=> grPr[:,2])
nearIndx(v,x) = findmin(abs.(v.-x))[2]
for i in 1:nrow(data)
    if ismissing(data[i, :Price])
        data[i, :Price] = d[data[i, :Grade]]
    end
    if ismissing(data[i, :Grade])
        data[i, :Grade] = grPr[nearIndx(grPr[:,2], data[i, :Price]), :Grade]
    end
end

println(first(data,5),"\n")
println(describe(data))


5×4 DataFrame
 Row │ Name       Date        Grade     Price  
     │ String15?  String15?   String1?  Int64? 
─────┼─────────────────────────────────────────
   1 │ MARYANNA   14/09/2008  A          79700
   2 │ REBECCA    11/03/2008  B          60000
   3 │ ASHELY     5/08/2008   E          24311
   4 │ KHADIJAH   2/09/2008   D          38904
   5 │ TANJA      1/12/2008   C          47052

4×7 DataFrame
 Row │ variable  mean     min        median   max        nmissing  eltype                   
     │ Symbol    Union…   Any        Union…   Any        Int64     Union                    
─────┼──────────────────────────────────────────────────────────────────────────────────────
   1 │ Name               ABBEY               ZACHARY           0  Union{Missing, String15}
   2 │ Date               1/07/2008           9/10/2008         0  Union{Missing, String15}
   3 │ Grade              A                   E                 0  Union{Missing, String1}
   4 │ Price     40037.9  8257       3